In [ ]:
from PIL import Image
import numpy as np
import cv2
import torch
import requests
import matplotlib.pyplot as plt

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
image

In [17]:
def pil2cv(image: Image) -> np.ndarray:
    mode = image.mode
    new_image: np.ndarray
    if mode == '1':
        new_image = np.array(image, dtype=np.uint8)
        new_image *= 255
    elif mode == 'L':
        new_image = np.array(image, dtype=np.uint8)
    elif mode == 'LA' or mode == 'La':
        new_image = np.array(image.convert('RGBA'), dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    elif mode == 'RGB':
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif mode == 'RGBA':
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    elif mode == 'LAB':
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_LAB2BGR)
    elif mode == 'HSV':
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_HSV2BGR)
    elif mode == 'YCbCr':
        # XXX: not sure if YCbCr == YCrCb
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_YCrCb2BGR)
    elif mode == 'P' or mode == 'CMYK':
        new_image = np.array(image.convert('RGB'), dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif mode == 'PA' or mode == 'Pa':
        new_image = np.array(image.convert('RGBA'), dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    else:
        raise ValueError(f'unhandled image color mode: {mode}')

    return new_image

In [18]:
repo = "intel-isl/MiDaS"
model_type = "MiDaS_small"
# Zoe_N

model_midas = torch.hub.load(repo, model_type)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
midas = model_midas.to(DEVICE)
midas = midas.eval()


In [ ]:
# sudo apt update && sudo apt install ffmpeg libsm6 libxext6 -y
midas_transforms = torch.hub.load(repo, "transforms")
transform = midas_transforms.small_transform


In [ ]:
#img = pil2cv(image)
img = np.array(image.convert("RGB"))
input_batch = transform(img).to(DEVICE)
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()


In [ ]:
plt.imshow(output)

In [ ]:

jetbot_ip = '194.47.156.140' 
url = f'http://{jetbot_ip}:8080/camera'
image = Image.open(requests.get(url, stream=True).raw)
image

In [32]:
img = np.array(image.convert("RGB"))
input_batch = transform(img).to(DEVICE)
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()


In [ ]:
fig, axes = plt.subplots(1,2, figsize=(21, 14))

axes[0].imshow(output)
axes[1].imshow(image)